## AI-HUB 데이터 확인

In [1]:
import os
from glob import glob

# 1. 모든 json 파일 경로 수집 (재귀적으로)
json_paths = glob('./data_validation/meta_json/**/*.json', recursive=True)

# 2. 파일 이름만 추출 (중복 제거용)
json_filenames = set(os.path.basename(path) for path in json_paths)

# 3. 개수 출력
print(f"📄 총 JSON 파일 수: {len(json_paths)}")
print(f"🆔 유니크한 JSON 파일 이름 수: {len(json_filenames)}")


📄 총 JSON 파일 수: 2700
🆔 유니크한 JSON 파일 이름 수: 2700


### step01 : json(메타)과 실제 데이터 확인

In [2]:
import json
from glob import glob
import os
import pandas as pd

# JSON 파일 경로 수집
json_paths = glob('./data_validation/meta_json/**/*.json', recursive=True)

records = []

for path in json_paths:
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        
        # 안전한 접근을 위한 get
        meta = data.get('metadata', {})
        animal = meta.get('animal', {})
        owner = meta.get('owner', {})
        inspect = meta.get('inspect', {})

        records.append({
            "json_file": os.path.basename(path),
            "breed": animal.get("breed"),
            "cat_action": inspect.get("action"),
            "cat_emotion": inspect.get("emotion"),
            "owner_situation": owner.get("situation"),
            "owner_emotion": owner.get("emotion"),
            # "seq": meta.get("seq"),
            "freams_anotations" : len(data['annotations']),
        })

# DataFrame 변환
df = pd.DataFrame(records)


In [3]:
# 미리보기
print(df.head())

                  json_file    breed cat_action cat_emotion  \
0  cat-grooming-052361.json  코리안 숏헤어   그루밍하는 동작       편안/안정   
1  cat-grooming-044680.json  코리안 숏헤어   그루밍하는 동작       편안/안정   
2  cat-grooming-025736.json     페르시안   그루밍하는 동작       편안/안정   
3  cat-grooming-028194.json     스노우슈   그루밍하는 동작       편안/안정   
4  cat-grooming-053718.json    노르웨이숲   그루밍하는 동작       편안/안정   

                        owner_situation owner_emotion  freams_anotations  
0  휴식시간, 자신만의 공간에 들어갔을 때(캔넬, 소파 침대 밑 등)         편안/안정                 79  
1                       산책 준비 또는 산책중일 때        행복/즐거움                 81  
2                     잠들기 전이나 같이 누워있을 때         편안/안정                 93  
3                                    기타        행복/즐거움                 99  
4                                    기타         편안/안정                114  


In [4]:
import os
import pandas as pd
from glob import glob

# # 메타데이터 파일 로딩
# df = pd.read_csv("your_meta_file.csv")  # <

# 루트 경로
base_path = "./data_validation/test_image"

# 결과를 저장할 리스트
found_list = []
frame_count_list = []
file_path_list = []

# 각 json_file에 대해 처리
for json_file in df['json_file']:
    folder_name = json_file.replace(".json", "")  # → cat-grooming-XXXXX
    found = False
    frame_count = 0
    found_path = None  # 폴더 경로 저장용

    # data_image/*/ 하위 폴더 탐색
    for action_dir in os.listdir(base_path):
        action_path = os.path.join(base_path, action_dir)

        # 디렉토리인지 확인
        if not os.path.isdir(action_path):
            continue

        target_path = os.path.join(action_path, folder_name)
        if os.path.isdir(target_path):
            found = True
            frame_files = glob(os.path.join(target_path, "*.jpg"))
            frame_count = len(frame_files)
            found_path = target_path  # 경로 저장
            break  # 찾았으면 중단

    found_list.append(found)
    frame_count_list.append(frame_count)
    file_path_list.append(found_path if found else None)

# 결과 추가
df["folder_found"] = found_list
df["frame_count_checked"] = frame_count_list
df["file_path"] = file_path_list

# 출력 예시
# print(df[["json_file", "folder_found", "frame_count_checked", "file_path"]].head())

In [5]:
df_true = df[df["folder_found"] == True]
print("✅ 폴더가 존재하는 항목 (True):")
print(df_true[["json_file", "frame_count_checked"]])

✅ 폴더가 존재하는 항목 (True):
                                 json_file  frame_count_checked
0                 cat-grooming-052361.json                   79
1                 cat-grooming-044680.json                   81
2                 cat-grooming-025736.json                   93
3                 cat-grooming-028194.json                   99
4                 cat-grooming-053718.json                  114
...                                    ...                  ...
2695               cat-getdown-017805.json                   84
2696               cat-getdown-066894.json                   80
2697               cat-getdown-040955.json                   63
2698  20201201_cat-getdown-009528.mp4.json                   88
2699               cat-getdown-037813.json                   88

[2700 rows x 2 columns]


In [6]:
df_false = df[df["folder_found"] == False]
print("❌ 폴더가 없는 항목 (False):")
print(df_false[["json_file"]])

❌ 폴더가 없는 항목 (False):
Empty DataFrame
Columns: [json_file]
Index: []


---

## Step02 : 매핑

In [7]:
df["frame_count_checked"].describe()

count    2700.00000
mean       99.24000
std        20.48396
min        15.00000
25%        84.00000
50%        95.00000
75%       110.00000
max       174.00000
Name: frame_count_checked, dtype: float64

In [8]:
print("🎯 cat_action 라벨 분포:")
print(df["cat_action"].value_counts())

🎯 cat_action 라벨 분포:
cat_action
그루밍하는 동작             638
꼬리를 흔드는 동작           386
앞발을 뻗어 휘적거리는 동작      368
걷거나 달리는 동작           306
옆으로 눕는 동작            213
발을 숨기고 웅크리고 앉는 동작    183
납작 엎드리는 동작           134
머리를 들이대는 동작          109
배를 보여주는 동작           104
앞발로 꾹꾹 누르는 동작         97
좌우로 뒹구는 동작            84
허리를 아치로 세우는 동작        23
걷거나 뛰는 동작             17
배를 보이는 동작             11
그루밍함                  10
납작 엎드림                 5
옆으로 누워 있음              4
꼬리를 흔든다                3
배를 보임                  3
팔을 뻗어 휘적거림             1
발을 숨기고 웅크리고 앉음         1
Name: count, dtype: int64


In [9]:
print("😊 cat_emotion 라벨 분포:")
print(df["cat_emotion"].value_counts())

# >> 불안/슬픔 + 공포로 라벨 합치기
# 6개에서 5개

😊 cat_emotion 라벨 분포:
cat_emotion
편안/안정     1935
행복/즐거움     495
공격성        146
화남/불쾌       80
불안/슬픔       36
공포           8
Name: count, dtype: int64


In [10]:
print("🏠 owner_situation 라벨 분포:")
print(df["owner_situation"].value_counts())

🏠 owner_situation 라벨 분포:
owner_situation
먹을것, 장난감이 앞에 있을 때                       658
기타                                      612
휴식시간, 자신만의 공간에 들어갔을 때(캔넬, 소파 침대 밑 등)    570
잠들기 전이나 같이 누워있을 때                       348
편안히 쓰다듬어 줄 때                            174
보호자가 집에 돌아왔을 때                          127
산책이나 노즈워크 중                              56
싫어하는 부위를 만질 때                            33
산책 준비 또는 산책중일 때                          17
밥그릇, 장난감과 같은 소유물을 만질 때                   17
다른 사람이나 동물을 만났을 때                        17
낯선 소리가 나거나 낯선 사람을 봤을 때                   15
낯선 동물 또는 사람을 만났을 때                       13
빗질/발톱깍기/목욕 등 위생관리를 할 때                    9
낯선 장소에 있거나 낯선 소리가 날 때                     8
다른 동물을 보거나 낯선 사람을 만날 때 산책 나왔을 때           5
낯선 소리가 났을 때                               4
목욕하거나 싫어하는 부위를 만질 때                       3
다른 동물을 보거나 낯선 사람을 만날 때                    3
혼날 때                                      3
보호자와 떨어질 때/혼자 남겨지거나 낯선장소에 있을 때            3
목욕할 때                              

In [11]:
unique_count_action = df["cat_action"].nunique()
unique_count_emotion = df["cat_emotion"].nunique()
unique_count_situation = df["owner_situation"].nunique()

print("cat_action의 유니크 라벨 수:", unique_count_action)
print("cat_emotion의 유니크 라벨 수:", unique_count_emotion)
print("owner_situation의 유니크 라벨 수:", unique_count_situation)

cat_action의 유니크 라벨 수: 21
cat_emotion의 유니크 라벨 수: 6
owner_situation의 유니크 라벨 수: 26


In [12]:
# 감정 통합 맵
emotion_map = {
    "불안/슬픔": "불안/공포",
    "공포": "불안/공포",
    "공격성" : "공격성",
    "편안/안정" : "편안/안정",
    "행복/즐거움" : "행복/즐거움",
    "화남/불쾌" : "화남/불쾌",
}

action_map = {
    "걷거나 뜀": "걷거나 달리는 동작",
    "걷거나 달리는 동작": "걷거나 달리는 동작",

    "꼬리를 흔든다": "꼬리를 흔드는 동작",
    "꼬리를 흔드는 동작": "꼬리를 흔드는 동작",

    "납작 엎드림": "납작 엎드리는 동작",
    "납작 엎드리는 동작": "납작 엎드리는 동작",

    "배를 보임": "배를 보여주는 동작",
    "배를 보여주는 동작": "배를 보여주는 동작",

    "그루밍함": "그루밍하는 동작",
    "그루밍하는 동작": "그루밍하는 동작",

    "머리를 들이댐": "머리를 들이대는 동작",
    "머리를 들이대는 동작": "머리를 들이대는 동작",

    "발을 숨기고 웅크리고 앉음": "발을 숨기고 웅크리고 앉는 동작",
    "발을 숨기고 웅크리고 앉는 동작": "발을 숨기고 웅크리고 앉는 동작",

    "앞발로 꾹꾹 누름": "앞발로 꾹꾹 누르는 동작",
    "앞발로 꾹꾹 누르는 동작": "앞발로 꾹꾹 누르는 동작",

    "옆으로 누워 있음": "옆으로 눕는 동작",
    "옆으로 눕는 동작": "옆으로 눕는 동작",

    "좌우로 뒹굴음": "좌우로 뒹구는 동작",
    "좌우로 뒹구는 동작": "좌우로 뒹구는 동작",

    "팔을 뻗어 휘적거림": "팔을 뻗어 휘적거리는 동작",
    "앞발을 뻗어 휘적거리는 동작": "팔을 뻗어 휘적거리는 동작",

    "허리를 아치로 세움": "허리를 아치로 세우는 동작",
    "허리를 아치로 세우는 동작": "허리를 아치로 세우는 동작"
        }

owner_situation_map = {
    # 🧸 놀이/자극 상황
    "먹을것, 장난감이 앞에 있을 때": "놀이/자극",
    "밥그릇, 장난감과 같은 소유물을 만질 때": "놀이/자극",
    "산책이나 노즈워크 중": "놀이/자극",
    "산책 준비 또는 산책중일 때": "놀이/자극",
    "다른 동물을 보거나 낯선 사람을 만날 때 산책 나왔을 때": "놀이/자극",

    # 😴 휴식/편안한 상황
    "휴식시간, 자신만의 공간에 들어갔을 때(캔넬, 소파 침대 밑 등)": "휴식/편안",
    "잠들기 전이나 같이 누워있을 때": "휴식/편안",
    "편안히 쓰다듬어 줄 때": "휴식/편안",

    # 👋 돌아옴/재회 상황
    "보호자가 집에 돌아왔을 때": "돌아옴/재회",

    # 🧼 위생관리
    "빗질/발톱깍기/목욕 등 위생관리를 할 때": "위생관리",
    "목욕할 때": "위생관리",
    "목욕하거나 싫어하는 부위를 만질 때": "위생관리",

    # 😾 싫은 자극/처벌
    "싫어하는 부위를 만질 때": "싫은 자극/처벌",
    "보호자에게 혼났을 때": "싫은 자극/처벌",
    "혼날 때": "싫은 자극/처벌",

    # 😨 낯선 상황/불안
    "낯선 소리가 나거나 낯선 사람을 봤을 때": "낯선 상황/불안",
    "낯선 장소에 있거나 낯선 소리가 날 때": "낯선 상황/불안",
    "낯선 소리가 났을 때": "낯선 상황/불안",
    "낯선 동물 또는 사람을 만났을 때": "낯선 상황/불안",
    "다른 사람이나 동물을 만났을 때": "낯선 상황/불안",
    "낯선 도구를 자신의 몸에 사용할 때(미용도구 등)": "낯선 상황/불안",
    "낯선 도구를 자신의 몸에 사용할 때(미용도구)": "낯선 상황/불안",
    "초인종 소리가 났을 때": "낯선 상황/불안",
    "다른 동물을 보거나 낯선 사람을 만날 때" : "낯선 상황/불안",

    # 😿 분리/고립
    "보호자와 떨어질 때/혼자 남겨지거나 낯선장소에 있을 때": "분리/고립",
    "보호자와 떨어지거나 혼자 남겨졌을 때": "분리/고립",
    
    "기타": "기타",
}


In [13]:
df["owner_situation_mapped"] = df["owner_situation"].map(owner_situation_map)
df["cat_emotion_mapped"] = df["cat_emotion"].map(emotion_map)
df["cat_action_mapped"] = df["cat_action"].map(action_map)


In [14]:
print("🎯 cat_action 라벨 분포:")
unique_count = df["cat_action_mapped"].nunique()

print(df["cat_action_mapped"].value_counts())
print("🐱 cat_action_mapped 유니크 라벨 수:", unique_count)

🎯 cat_action 라벨 분포:
cat_action_mapped
그루밍하는 동작             648
꼬리를 흔드는 동작           389
팔을 뻗어 휘적거리는 동작       369
걷거나 달리는 동작           306
옆으로 눕는 동작            217
발을 숨기고 웅크리고 앉는 동작    184
납작 엎드리는 동작           139
머리를 들이대는 동작          109
배를 보여주는 동작           107
앞발로 꾹꾹 누르는 동작         97
좌우로 뒹구는 동작            84
허리를 아치로 세우는 동작        23
Name: count, dtype: int64
🐱 cat_action_mapped 유니크 라벨 수: 12


In [15]:
print("🎯 cat_emotion 라벨 분포:")
unique_count = df["cat_emotion_mapped"].nunique()

print(df["cat_emotion_mapped"].value_counts())
print("🐱 cat_emotion_mapped 유니크 라벨 수:", unique_count)

🎯 cat_emotion 라벨 분포:
cat_emotion_mapped
편안/안정     1935
행복/즐거움     495
공격성        146
화남/불쾌       80
불안/공포       44
Name: count, dtype: int64
🐱 cat_emotion_mapped 유니크 라벨 수: 5


In [16]:
print("🎯 owner_situation_mapped 라벨 분포:")
unique_count = df["owner_situation_mapped"].nunique()

print(df["owner_situation_mapped"].value_counts())
print("🐱 owner_situation_mapped 유니크 라벨 수:", unique_count)

🎯 owner_situation_mapped 라벨 분포:
owner_situation_mapped
휴식/편안       1092
놀이/자극        753
기타           612
돌아옴/재회       127
낯선 상황/불안      62
싫은 자극/처벌      37
위생관리          13
분리/고립          3
Name: count, dtype: int64
🐱 owner_situation_mapped 유니크 라벨 수: 8


---

In [17]:
df.file_path

0       ./data_validation/test_image/GROOMING/cat-groo...
1       ./data_validation/test_image/GROOMING/cat-groo...
2       ./data_validation/test_image/GROOMING/cat-groo...
3       ./data_validation/test_image/GROOMING/cat-groo...
4       ./data_validation/test_image/GROOMING/cat-groo...
                              ...                        
2695    ./data_validation/test_image/GETDOWN/cat-getdo...
2696    ./data_validation/test_image/GETDOWN/cat-getdo...
2697    ./data_validation/test_image/GETDOWN/cat-getdo...
2698    ./data_validation/test_image/GETDOWN/20201201_...
2699    ./data_validation/test_image/GETDOWN/cat-getdo...
Name: file_path, Length: 2700, dtype: object

In [18]:
df['file_path'] = df['file_path'].str.replace('./data_validation/test_image/', '', regex=False)

In [19]:
df.file_path

0                  GROOMING/cat-grooming-052361
1                  GROOMING/cat-grooming-044680
2                  GROOMING/cat-grooming-025736
3                  GROOMING/cat-grooming-028194
4                  GROOMING/cat-grooming-053718
                         ...                   
2695                 GETDOWN/cat-getdown-017805
2696                 GETDOWN/cat-getdown-066894
2697                 GETDOWN/cat-getdown-040955
2698    GETDOWN/20201201_cat-getdown-009528.mp4
2699                 GETDOWN/cat-getdown-037813
Name: file_path, Length: 2700, dtype: object

In [22]:
# 사용할 칼럼 리스트
cols = ['json_file', 'file_path', 'frame_count_checked', 'cat_action_mapped', 'cat_emotion_mapped', 'owner_situation_mapped']

# 원하는 칼럼만 추출
df_final = df[cols].copy()

# 컬럼명 변경
df_final.rename(columns={
    'json_file': 'meta_json',
    'frame_count_checked': 'number of frames',
    'cat_action_mapped' : 'cat_action',
    'cat_emotion_mapped': 'cat_emotion',
    'owner_situation_mapped': 'owner_situation'
}, inplace=True)

# CSV 저장 (경로와 파일명 원하는 대로 수정)
df_final.to_csv('./data_fin/test_dataset.csv', index=False)

# 저장 확인 출력
print(df_final.head())


                  meta_json                     file_path  number of frames  \
0  cat-grooming-052361.json  GROOMING/cat-grooming-052361                79   
1  cat-grooming-044680.json  GROOMING/cat-grooming-044680                81   
2  cat-grooming-025736.json  GROOMING/cat-grooming-025736                93   
3  cat-grooming-028194.json  GROOMING/cat-grooming-028194                99   
4  cat-grooming-053718.json  GROOMING/cat-grooming-053718               114   

  cat_action cat_emotion owner_situation  
0   그루밍하는 동작       편안/안정           휴식/편안  
1   그루밍하는 동작       편안/안정           놀이/자극  
2   그루밍하는 동작       편안/안정           휴식/편안  
3   그루밍하는 동작       편안/안정              기타  
4   그루밍하는 동작       편안/안정              기타  


### 확인용

In [23]:
import pandas as pd

# CSV 파일 불러오기
df_v = pd.read_csv('./data_fin/test_dataset.csv')

# meta_json 개수 (전체 파일 수)
print("== meta_json 개수 (전체 파일 수) ==")
print(len(df_v['meta_json'].unique()))
print()

# number of frames 통계
print("== number of frames 통계 ==")
print(df_v['number of frames'].describe())
print()

# cat_action 분포 및 유니크 개수
print("== cat_action 분포 ==")
print(df_v['cat_action'].value_counts())
print(f"🐾 cat_action 라벨 개수: {df_v['cat_action'].nunique()}")
print()

# cat_emotion 분포 및 유니크 개수
print("== cat_emotion 분포 ==")
print(df_v['cat_emotion'].value_counts())
print(f"❤️cat_emotion 라벨 개수: {df_v['cat_emotion'].nunique()}")
print()

# owner_situation 분포 및 유니크 개수
print("== owner_situation 분포 ==")
print(df_v['owner_situation'].value_counts())
print(f"🏠 owner_situation 라벨 개수: {df_v['owner_situation'].nunique()}")
print()

== meta_json 개수 (전체 파일 수) ==
2700

== number of frames 통계 ==
count    2700.00000
mean       99.24000
std        20.48396
min        15.00000
25%        84.00000
50%        95.00000
75%       110.00000
max       174.00000
Name: number of frames, dtype: float64

== cat_action 분포 ==
cat_action
그루밍하는 동작             648
꼬리를 흔드는 동작           389
팔을 뻗어 휘적거리는 동작       369
걷거나 달리는 동작           306
옆으로 눕는 동작            217
발을 숨기고 웅크리고 앉는 동작    184
납작 엎드리는 동작           139
머리를 들이대는 동작          109
배를 보여주는 동작           107
앞발로 꾹꾹 누르는 동작         97
좌우로 뒹구는 동작            84
허리를 아치로 세우는 동작        23
Name: count, dtype: int64
🐾 cat_action 라벨 개수: 12

== cat_emotion 분포 ==
cat_emotion
편안/안정     1935
행복/즐거움     495
공격성        146
화남/불쾌       80
불안/공포       44
Name: count, dtype: int64
❤️cat_emotion 라벨 개수: 5

== owner_situation 분포 ==
owner_situation
휴식/편안       1092
놀이/자극        753
기타           612
돌아옴/재회       127
낯선 상황/불안      62
싫은 자극/처벌      37
위생관리          13
분리/고립          3
Name: count, dtype: int

---